In [1]:
from chromedriver_py import binary_path
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.firefox.service import Service as FirefoxService
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.action_chains import ActionChains
import multiprocessing as mp
import istarmap  # import to apply patch

from joblib import Parallel, delayed

import tqdm

import numpy as np
import pandas as pd

from datetime import datetime

from PIL import Image
from urllib.request import urlopen,Request
from io import BytesIO

import time
import psutil
import os

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

import warnings
warnings.filterwarnings('ignore')

In [2]:
ruta = os.getcwd()

In [3]:
ruta

'/Users/cevalenciam/Library/CloudStorage/OneDrive-Personal/SmartBuy/smartbuy/web-scraper-jumbo'

In [11]:
def get_driver():
    driver = webdriver.Firefox(service=FirefoxService('../geckodriver'))
    driver.maximize_window()
    return driver

In [12]:
# def get_driver():
#     chrome_options = Options()
#     # chrome_options.add_argument("--headless")
#     chrome_options.add_argument("--no-sandbox")
#     chrome_options.add_argument("--disable-gpu")
#     # chrome_options.add_argument("--remote-debugging-port=9222")
#     chrome_options.add_argument("--disable-dev-shm-usage")
#     chrome_options.add_argument("--disable-extensions")
#     chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
#     chrome_options.add_experimental_option('useAutomationExtension', False)
#     chrome_options.add_experimental_option('extensionLoadTimeout', 300000)
#     prefs = {"profile.default_content_setting_values.notifications" : 2}
#     chrome_options.add_experimental_option("prefs",prefs)
#     s = ChromeService(binary_path)
#     driver = webdriver.Chrome(service=s, options=chrome_options)
#     driver.maximize_window()
#     return driver

In [13]:
def get_url_jumbo():
    
    url_jumbo = 'https://www.jumbo.cl'

    return(url_jumbo)

In [14]:
def get_info_category():

    chrome_options = Options()
    # chrome_options.add_argument("--headless")
    prefs = {"profile.default_content_setting_values.notifications" : 2}
    chrome_options.add_experimental_option("prefs",prefs)

    s = ChromeService(binary_path)
    driver = webdriver.Chrome(service=s, options=chrome_options)

    driver.maximize_window()

    #Vamos a la dirección web de la página objetivo
    driver.get(get_url_jumbo())

    list_categories = []

    #Espera de 3 seg
    time.sleep(3)

    #Cerrar pop ups
    try:
        boton = wait(driver, 4).until(ec.element_to_be_clickable((By.XPATH, "/html/body/div[@id='root']/div[@class='app-content']/header[@class='new-header-v2']/div[@class='header-content-v2 ']/div[@class='content-menu-wrapper-v2 ']/div[@class='popover-container undefined']/div[@class='popover-select-delivery-method-v2']/div[@class='popover-delivery-info']/button[@class='popover-delivery-close jumbo-icon-new-close']")))
        boton.click()
    except TimeoutException:
        print("Pop up no encontrado")

    #Click en categorías
    try:
        boton = wait(driver, 4).until(ec.element_to_be_clickable((By.XPATH, "/html/body/div[@id='root']/div[@class='app-content']/header[@class='new-header-v2']/div[@class='header-content-v2 ']/div[@class='main-categories-offers-wrapper']/div[@class='content-categories-offers-wrapper']/div[@class='categories-offers-header']/button[@class='categories-dropdown-button']")))
        boton.click()
    except TimeoutException:
        print("Botón categorías no encontrado")

    list_categories = []

    #Obtener las categorías
    try:
        wait(driver, 8).until(ec.element_to_be_clickable((By.XPATH, "/html/body/div[@id='root']/div[@class='app-content']/header[@class='new-header-v2']/div[@class='header-content-v2 ']/div[@class='main-categories-offers-wrapper']/div[@class='content-categories-offers-wrapper']/div[@class='categories-offers-header']/div[@class='categories-menu-container']/div[@class='categories-menu-dropdown']/ul[@class='categories-menu-list']/li/a")))
        first_level_categories = driver.find_elements("xpath", "/html/body/div[@id='root']/div[@class='app-content']/header[@class='new-header-v2']/div[@class='header-content-v2 ']/div[@class='main-categories-offers-wrapper']/div[@class='content-categories-offers-wrapper']/div[@class='categories-offers-header']/div[@class='categories-menu-container']/div[@class='categories-menu-dropdown']/ul[@class='categories-menu-list']/li/a")
        
        for first, i in zip(first_level_categories, range(len(first_level_categories))):
            
            first_level_category = first.text
            url_first_level_category = first.get_attribute("href")

            ActionChains(driver).move_to_element(first).perform()

            wait(driver, 8).until(ec.presence_of_element_located((By.XPATH, "/html/body/div[@id='root']/div[@class='app-content']/header[@class='new-header-v2']/div[@class='header-content-v2 ']/div[@class='main-categories-offers-wrapper']/div[@class='content-categories-offers-wrapper']/div[@class='categories-offers-header']/div[@class='categories-menu-container']/div[@class='categories-menu-dropdown']/div[@class='sub-categories-container']/ul[@class='sub-categories-wrapper']/li[@class='sub-categories-menu-list']")))
            second_level_categories = driver.find_elements("xpath", "/html/body/div[@id='root']/div[@class='app-content']/header[@class='new-header-v2']/div[@class='header-content-v2 ']/div[@class='main-categories-offers-wrapper']/div[@class='content-categories-offers-wrapper']/div[@class='categories-offers-header']/div[@class='categories-menu-container']/div[@class='categories-menu-dropdown']/div[@class='sub-categories-container']/ul[@class='sub-categories-wrapper']/li[@class='sub-categories-menu-list']")

            for second in second_level_categories:

                try:
                    second_level_category = second.find_elements("xpath", "a[@class='sub-categories-title']")[0].text
                    url_second_level_category = second.find_elements("xpath", "a[@class='sub-categories-title']")[0].get_attribute("href")
        
                except:
                    print("Segundas categorias no encontradas")
                    second_level_category = ''
                    url_second_level_category = ''
                
                try:
                    third_level_categories = second.find_elements("xpath", "a[@class='sub-categories-title-childrens']")

                    for third in third_level_categories:

                        third_level_category = third.text
                        url_third_level_category = third.get_attribute("href")

                        list_categories.append([first_level_category, url_first_level_category, second_level_category, url_second_level_category, third_level_category, url_third_level_category])
                
                except:
                    print("Terceras categorias no encontradas")
                    third_level_category = ''
                    url_third_level_category = ''
                    list_categories.append([first_level_category, url_first_level_category, second_level_category, url_second_level_category, third_level_category, url_third_level_category])
                
                try:
                    third_level_category = second.find_elements("xpath", "a[@class='sub-categories-link-show-all']")[0].text
                    url_third_level_category = second.find_elements("xpath", "a[@class='sub-categories-link-show-all']")[0].get_attribute("href")
                    
                    list_categories.append([first_level_category, url_first_level_category, second_level_category, url_second_level_category, third_level_category, url_third_level_category])
                
                except:
                    print("Ver todos no encontrado")

    except TimeoutException:
        print("Categorias no encontradas")

    driver.close()

    return(pd.DataFrame(list_categories, columns = ['first_level_category', 'url_first_level_category', 'second_level_category', 'url_second_level_category', 'third_level_category', 'url_third_level_category']), len(list_categories))

In [15]:
df_url_cat, num_cat = get_info_category()

In [16]:
num_cat

312

In [17]:
actual_date = str(datetime.now())[0:10]

df_url_cat.to_excel(ruta + f'/df_categorias_jumbo.xlsx'.format(actual_date), index=0)

In [18]:
def get_pages_category(pais:str, i:str, first:str, second:str, third:str):

    driver = get_driver()

    total_url_cat = list()

    driver.get(i)

    time.sleep(3)

    #Cerrar pop ups
    try:
        boton = wait(driver, 4).until(ec.element_to_be_clickable((By.XPATH, "/html/body/div[@id='root']/div[@class='app-content']/header[@class='new-header-v2']/div[@class='header-content-v2 ']/div[@class='content-menu-wrapper-v2 ']/div[@class='popover-container undefined']/div[@class='popover-select-delivery-method-v2']/div[@class='popover-delivery-info']/button[@class='popover-delivery-close jumbo-icon-new-close']")))
        boton.click()
    except TimeoutException:
        print("Pop up no encontrado")

    per_page = 40

    try:

        wait(driver, 4).until(ec.visibility_of_element_located((By.XPATH, "/html/body/div[@id='root']/div[@class='app-content']/div[@class='catalog-page page']/div[@class='container']/div[@class='catalog-page-content']/main[@class='shelf-main']/div[@class='shelf-header']/div[@class='shelf-header-title-wrapper']/div[@class='title-with-bar title-with-bar-medium shelf-header-title']/div[@class='title-with-bar-wrapper']/span[@class='title-with-bar-aditional-text']")))
        count = int(driver.find_elements("xpath", "/html/body/div[@id='root']/div[@class='app-content']/div[@class='catalog-page page']/div[@class='container']/div[@class='catalog-page-content']/main[@class='shelf-main']/div[@class='shelf-header']/div[@class='shelf-header-title-wrapper']/div[@class='title-with-bar title-with-bar-medium shelf-header-title']/div[@class='title-with-bar-wrapper']/span[@class='title-with-bar-aditional-text']")[0].text.replace(' productos', ''))
        n_paginas = int(np.ceil(count/per_page))
        
        if n_paginas>1:
            url_cat_page = [[i + '?page=' + str(j), first, second, third] for j in range(2, n_paginas+1)]
            url_cat_page_2 = [[i + '&page=' + str(j), first, second, third] for j in range(2, n_paginas+1)]
            total_url_cat.append([i, first, second, third])
            total_url_cat.extend(url_cat_page)
            total_url_cat.extend(url_cat_page_2)
        else:
            total_url_cat.append([i, first, second, third])
    
    except:
        total_url_cat.append([i, first, second, third])
        
    driver.close()
    
    return(total_url_cat)

In [19]:
df_url_cat[(df_url_cat['first_level_category']=='Supermercado') & (df_url_cat['second_level_category']=='Bebé')]

,first_level_category,url_first_level_category,second_level_category,url_second_level_category,third_level_category,url_third_level_category
5,Supermercado,https://www.jumbo.cl/,Bebé,https://www.jumbo.cl/mi-bebe,Pañales y Toallas Húmedas,https://www.jumbo.cl/mi-bebe/panales-y-toallas...
6,Supermercado,https://www.jumbo.cl/,Bebé,https://www.jumbo.cl/mi-bebe,"Colados, Picados y Otros",https://www.jumbo.cl/mi-bebe/colados-picados-y...
7,Supermercado,https://www.jumbo.cl/,Bebé,https://www.jumbo.cl/mi-bebe,Leche y Suplementos Infantiles,https://www.jumbo.cl/mi-bebe/leche-y-suplement...
8,Supermercado,https://www.jumbo.cl/,Bebé,https://www.jumbo.cl/mi-bebe,Perfumería Infantil,https://www.jumbo.cl/mi-bebe/perfumeria-infantil
9,Supermercado,https://www.jumbo.cl/,Bebé,https://www.jumbo.cl/mi-bebe,Mostrar Todo,https://www.jumbo.cl/mi-bebe


In [20]:
(df_url_cat.url_third_level_category[0], df_url_cat.first_level_category[0], df_url_cat.second_level_category[0], df_url_cat.third_level_category[0])

('https://www.jumbo.cl/cuisine-co?nombre_promo=menu-cuisine-co-25092023',
 'Supermercado',
 'Exclusivo en Jumbo',
 'Cuisine & Co')

In [21]:
pais='CL'

In [22]:
get_pages_category('CL', df_url_cat.url_third_level_category[0], df_url_cat.first_level_category[0], df_url_cat.second_level_category[0], df_url_cat.third_level_category[0])

[['https://www.jumbo.cl/cuisine-co?nombre_promo=menu-cuisine-co-25092023',
  'Supermercado',
  'Exclusivo en Jumbo',
  'Cuisine & Co']]

In [ ]:
df_url_cat[(df_url_cat['first_level_category']=='Supermercado') &
            (df_url_cat['second_level_category']=='Vinos, Cervezas y Licores')]


,first_level_category,url_first_level_category,second_level_category,url_second_level_category,third_level_category,url_third_level_category
25,Supermercado,https://www.jumbo.cl/,"Vinos, Cervezas y Licores",https://www.jumbo.cl/vinos-cervezas-y-licores,Vinos y Espumantes,https://www.jumbo.cl/vinos-cervezas-y-licores/...
26,Supermercado,https://www.jumbo.cl/,"Vinos, Cervezas y Licores",https://www.jumbo.cl/vinos-cervezas-y-licores,Cervezas,https://www.jumbo.cl/vinos-cervezas-y-licores/...
27,Supermercado,https://www.jumbo.cl/,"Vinos, Cervezas y Licores",https://www.jumbo.cl/vinos-cervezas-y-licores,Destilados,https://www.jumbo.cl/vinos-cervezas-y-licores/...
28,Supermercado,https://www.jumbo.cl/,"Vinos, Cervezas y Licores",https://www.jumbo.cl/vinos-cervezas-y-licores,Licores y Cócteles,https://www.jumbo.cl/vinos-cervezas-y-licores/...
29,Supermercado,https://www.jumbo.cl/,"Vinos, Cervezas y Licores",https://www.jumbo.cl/vinos-cervezas-y-licores,Mostrar Todo,https://www.jumbo.cl/vinos-cervezas-y-licores


In [ ]:
get_pages_category('CL', df_url_cat.url_third_level_category[25], df_url_cat.first_level_category[25], df_url_cat.second_level_category[25], df_url_cat.third_level_category[25])

[['https://www.jumbo.cl/vinos-cervezas-y-licores/vinos',
  'Supermercado',
  'Vinos, Cervezas y Licores',
  'Vinos y Espumantes'],
 ['https://www.jumbo.cl/vinos-cervezas-y-licores/vinos?page=2',
  'Supermercado',
  'Vinos, Cervezas y Licores',
  'Vinos y Espumantes'],
 ['https://www.jumbo.cl/vinos-cervezas-y-licores/vinos?page=3',
  'Supermercado',
  'Vinos, Cervezas y Licores',
  'Vinos y Espumantes'],
 ['https://www.jumbo.cl/vinos-cervezas-y-licores/vinos?page=4',
  'Supermercado',
  'Vinos, Cervezas y Licores',
  'Vinos y Espumantes'],
 ['https://www.jumbo.cl/vinos-cervezas-y-licores/vinos?page=5',
  'Supermercado',
  'Vinos, Cervezas y Licores',
  'Vinos y Espumantes'],
 ['https://www.jumbo.cl/vinos-cervezas-y-licores/vinos?page=6',
  'Supermercado',
  'Vinos, Cervezas y Licores',
  'Vinos y Espumantes'],
 ['https://www.jumbo.cl/vinos-cervezas-y-licores/vinos?page=7',
  'Supermercado',
  'Vinos, Cervezas y Licores',
  'Vinos y Espumantes'],
 ['https://www.jumbo.cl/vinos-cervezas-y-

In [ ]:
(df_url_cat['first_level_category']+df_url_cat['second_level_category']).nunique()

71

In [ ]:
def get_pages_categories(pais:str, df_categories:pd.DataFrame):

    list_res = list()

    results = Parallel(n_jobs=-1)(delayed(get_pages_category)(pais, i, first, second, third) for i, first, second, third in tqdm.tqdm(zip(df_categories.url_third_level_category, df_categories.first_level_category, df_categories.second_level_category, df_categories.third_level_category)))
    
    for res in results:
        list_res = list_res + res
    
    return(pd.DataFrame(list_res, columns=['url_category', 'first_level_category', 'second_level_category', 'third_level_category']).drop_duplicates().values.tolist())

In [ ]:
list_pages_categories = get_pages_categories('CL', df_url_cat.iloc[25:26, :])

1it [00:00, 53.38it/s]

In [ ]:
list_pages_categories

[['https://www.jumbo.cl/vinos-cervezas-y-licores/vinos',
  'Supermercado',
  'Vinos, Cervezas y Licores',
  'Vinos y Espumantes'],
 ['https://www.jumbo.cl/vinos-cervezas-y-licores/vinos?page=2',
  'Supermercado',
  'Vinos, Cervezas y Licores',
  'Vinos y Espumantes'],
 ['https://www.jumbo.cl/vinos-cervezas-y-licores/vinos?page=3',
  'Supermercado',
  'Vinos, Cervezas y Licores',
  'Vinos y Espumantes'],
 ['https://www.jumbo.cl/vinos-cervezas-y-licores/vinos?page=4',
  'Supermercado',
  'Vinos, Cervezas y Licores',
  'Vinos y Espumantes'],
 ['https://www.jumbo.cl/vinos-cervezas-y-licores/vinos?page=5',
  'Supermercado',
  'Vinos, Cervezas y Licores',
  'Vinos y Espumantes'],
 ['https://www.jumbo.cl/vinos-cervezas-y-licores/vinos?page=6',
  'Supermercado',
  'Vinos, Cervezas y Licores',
  'Vinos y Espumantes'],
 ['https://www.jumbo.cl/vinos-cervezas-y-licores/vinos?page=7',
  'Supermercado',
  'Vinos, Cervezas y Licores',
  'Vinos y Espumantes'],
 ['https://www.jumbo.cl/vinos-cervezas-y-

In [ ]:
list_pages_categories[0]

['https://www.jumbo.cl/vinos-cervezas-y-licores/vinos',
 'Supermercado',
 'Vinos, Cervezas y Licores',
 'Vinos y Espumantes']

In [ ]:
def get_info_product(pais:str, list_pages_category:list):

    driver = get_driver()

    url_category = list()
    name_product = list()
    url_product = list()
    first_level_category = list()
    second_level_category = list()
    third_level_category = list()

    driver.get(list_pages_category[0])

    time.sleep(3)

    #Cerrar pop ups
    try:
        boton = wait(driver, 4).until(ec.element_to_be_clickable((By.XPATH, "/html/body/div[@id='root']/div[@class='app-content']/header[@class='new-header-v2']/div[@class='header-content-v2 ']/div[@class='content-menu-wrapper-v2 ']/div[@class='popover-container undefined']/div[@class='popover-select-delivery-method-v2']/div[@class='popover-delivery-info']/button[@class='popover-delivery-close jumbo-icon-new-close']")))
        boton.click()
    except TimeoutException:
        print("Pop up no encontrado")

    try:

        wait(driver, 4).until(ec.element_to_be_clickable((By.XPATH, "/html/body/div[@id='root']/div[@class='app-content']/div[@class='catalog-page page']/div[@class='container']/div[@class='catalog-page-content']/main[@class='shelf-main']/div[@class='shelf-wrapper']/div[@class='shelf-products-wrap']/div[@class='shelf-content']/div[@class='product-card']/div[@class='product-card-wrap']")))
        sku = driver.find_elements("xpath", "/html/body/div[@id='root']/div[@class='app-content']/div[@class='catalog-page page']/div[@class='container']/div[@class='catalog-page-content']/main[@class='shelf-main']/div[@class='shelf-wrapper']/div[@class='shelf-products-wrap']/div[@class='shelf-content']/div[@class='product-card']/div[@class='product-card-wrap']")

        for j in sku:
            url_category.append(list_pages_category[0])

            wait(j, 4).until(ec.presence_of_element_located((By.XPATH, "a[@class='product-card-name']")))
            name_product.append(j.find_elements("xpath", "a[@class='product-card-name']")[0].text)

            wait(j, 4).until(ec.presence_of_element_located((By.XPATH, "a[@class='product-card-name']")))
            url_product.append(j.find_elements("xpath", "a[@class='product-card-name']")[0].get_attribute("href"))
            
            first_level_category.append(list_pages_category[1])
            second_level_category.append(list_pages_category[2])
            third_level_category.append(list_pages_category[3])

    except:
        print('ERROR: url not found')
        
    driver.close()

    df_url_sku = pd.DataFrame()

    df_url_sku['url_category'] = url_category
    df_url_sku['name_product'] = name_product
    df_url_sku['url_product'] = url_product
    df_url_sku['first_level_category'] = first_level_category
    df_url_sku['second_level_category'] = second_level_category
    df_url_sku['third_level_category'] = third_level_category

    return(df_url_sku.drop_duplicates())

In [ ]:
list_pages_categories[0]

['https://www.jumbo.cl/vinos-cervezas-y-licores/vinos',
 'Supermercado',
 'Vinos, Cervezas y Licores',
 'Vinos y Espumantes']

In [ ]:
for i in range(len(list_pages_categories)):
    get_info_product('CL', list_pages_categories[i]).head(3)

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=120.0.6099.234)
Stacktrace:
0   chromedriver_mac-x64                0x000000010de96cc8 chromedriver_mac-x64 + 4844744
1   chromedriver_mac-x64                0x000000010de8e243 chromedriver_mac-x64 + 4809283
2   chromedriver_mac-x64                0x000000010da5c77d chromedriver_mac-x64 + 411517
3   chromedriver_mac-x64                0x000000010da302a8 chromedriver_mac-x64 + 230056
4   chromedriver_mac-x64                0x000000010dade4df chromedriver_mac-x64 + 943327
5   chromedriver_mac-x64                0x000000010daf5486 chromedriver_mac-x64 + 1037446
6   chromedriver_mac-x64                0x000000010dad6a63 chromedriver_mac-x64 + 911971
7   chromedriver_mac-x64                0x000000010da9e1b3 chromedriver_mac-x64 + 680371
8   chromedriver_mac-x64                0x000000010da9f7ce chromedriver_mac-x64 + 686030
9   chromedriver_mac-x64    

In [ ]:
def get_info_products(pais:str, df_info_category:list):

    results = Parallel(n_jobs=-1, prefer="threads")(delayed(get_info_product)(pais, i) for i in tqdm.tqdm(df_info_category))
    
    if len(results)>0:
        return(pd.concat(results).drop_duplicates())
    elif len(results)==0:
        return(pd.DataFrame(columns=['url_category', 'name_product', 'url_product', 'first_level_category', 'second_level_category', 'third_level_category']))

In [ ]:
df_url_sku = get_info_products('CL', list_pages_categories[:5])

NameError: name 'list_pages_categories' is not defined

In [ ]:
df_url_sku.shape

(200, 6)

In [ ]:
df_url_sku.head(3)

,url_category,name_product,url_product,first_level_category,second_level_category,third_level_category
0,https://www.jumbo.cl/cuisine-co?nombre_promo=m...,Carne molida 5% grasa 500 g,https://www.jumbo.cl/carne-molida-5-materia-gr...,Supermercado,Exclusivo en Jumbo,Cuisine & Co
1,https://www.jumbo.cl/cuisine-co?nombre_promo=m...,Carne molida vacuno 10% grasa 500 g,https://www.jumbo.cl/carne-molida-10-materia-g...,Supermercado,Exclusivo en Jumbo,Cuisine & Co
2,https://www.jumbo.cl/cuisine-co?nombre_promo=m...,Carne molida magra 3% grasa 500 g,https://www.jumbo.cl/carne-molida-magra-3-gras...,Supermercado,Exclusivo en Jumbo,Cuisine & Co


In [ ]:
df_url_sku.iloc[1,2]

'https://www.jumbo.cl/carne-molida-10-materia-grasa/p'

In [ ]:
def validate_url_cat(total_url_sku:pd.DataFrame, df_categories:pd.DataFrame):

    cat_validate = total_url_sku[['url_category']].drop_duplicates()
    cat_validate['url_down'] = 0

    cat_validate = df_categories.merge(cat_validate.rename(columns={'url_category': 'url_third_level_category'}), how='left')
    cat_validate['url_down'] = cat_validate.url_down.fillna(1)

    return(cat_validate)

In [ ]:
validate_url_cat(df_url_sku, df_url_cat)

In [ ]:
def get_scraping_sku(pais_sku:str, url_sku:list):

    driver = get_driver()

    driver.get(url_sku[0])

    

    try:

        wait(driver, 64).until(ec.presence_of_element_located((By.XPATH, "/html/body/div[@id='root']/div[@class='app-content']/div[@class='product-page']/div[@class='product-container page']/div[@class='container']/main[@class='product-content']")))
        df_producto = driver.find_elements("xpath", "/html/body/div[@id='root']/div[@class='app-content']/div[@class='product-page']/div[@class='product-container page']/div[@class='container']/main[@class='product-content']")[0]

        try:
            wait(df_producto, 4).until(ec.presence_of_element_located((By.XPATH, "div[@class='product-info']/div[@class='product-info-wrapper']/div[@class='product-aditional-info']/div[@class='aditional-info']/span[@class='product-code']")))
            id_url = df_producto.find_elements("xpath", "div[@class='product-info']/div[@class='product-info-wrapper']/div[@class='product-aditional-info']/div[@class='aditional-info']/span[@class='product-code']")[0].text
        except:
            id_url = ''

        try: 
            wait(df_producto, 4).until(ec.presence_of_element_located((By.XPATH, "div[@class='product-info']/div[@class='product-info-wrapper']/div[@class='product-container-title']/h1")))
            name_url = df_producto.find_elements("xpath", "div[@class='product-info']/div[@class='product-info-wrapper']/div[@class='product-container-title']/h1")[0].text
        except:
            name_url = ''

        try:
            wait(df_producto, 4).until(ec.presence_of_element_located((By.XPATH, "div[@class='product-wrap-section activeNutricional']/div[@class='product-description']/div[@class='product-description-content']")))
            description_url =  df_producto.find_elements("xpath", "div[@class='product-wrap-section activeNutricional']/div[@class='product-description']/div[@class='product-description-content']")[0].text
        except:
            description_url = ''

        try:
            wait(df_producto, 4).until(ec.presence_of_element_located((By.XPATH, "div[@class='product-info']/div[@class='product-info-wrapper']/div[@class='product-aditional-info']/div[@class='aditional-info']/a")))
            atr_sku =  df_producto.find_elements("xpath", "div[@class='product-info']/div[@class='product-info-wrapper']/div[@class='product-aditional-info']/div[@class='aditional-info']/a")[0].text
        except:
            atr_sku = ''
            
        try:

            normal_price = '' 
            internet_price = ''
            cmr_price = ''
            
            try:
                wait(df_producto, 4).until(ec.presence_of_element_located((By.XPATH, "div[@class='product-info']/div[@class='product-info-wrapper']/div[@class='product-card-prices']/div[@class='prices-price prices-tpm']/span[@class='prices-old-price']")))
                normal_price = df_producto.find_elements("xpath", "div[@class='product-info']/div[@class='product-info-wrapper']/div[@class='product-card-prices']/div[@class='prices-price prices-tpm']/span[@class='prices-old-price']")[0].text
            
            except:
                normal_price = ''

            wait(df_producto, 4).until(ec.presence_of_element_located((By.XPATH, "div[@class='product-info']/div[@class='product-info-wrapper']/div[@class='product-card-prices']/div[@class='prices-price prices-tpm']/span[@class='prices-main-price']")))
            internet_price = df_producto.find_elements("xpath", "div[@class='product-info']/div[@class='product-info-wrapper']/div[@class='product-card-prices']/div[@class='prices-price prices-tpm']/span[@class='prices-main-price']")[0].text

        except:
            normal_price = ''
            internet_price = ''
            cmr_price = ''

        try:
            wait(df_producto, 4).until(ec.presence_of_element_located((By.XPATH, "div[@class='product-info']/div[@class='product-info-wrapper']/div[@class='product-aditional-info']/div[@class='aditional-info']/span[@class='product-code']")))
            id_client_sku = df_producto.find_elements("xpath", "div[@class='product-info']/div[@class='product-info-wrapper']/div[@class='product-aditional-info']/div[@class='aditional-info']/span[@class='product-code']")[0].text
        except:
            id_client_sku = ''

        try:
            wait(df_producto, 4).until(ec.presence_of_element_located((By.XPATH, "div[@class='product-info']/div[@class='product-info-wrapper']/div[@class='product-aditional-info']/div[@class='aditional-info']/span[@class='product-code']")))
            cod_sku = df_producto.find_elements("xpath", "div[@class='product-info']/div[@class='product-info-wrapper']/div[@class='product-aditional-info']/div[@class='aditional-info']/span[@class='product-code']")[0].text
        except:
            cod_sku = ''

        try:
            wait(df_producto, 4).until(ec.presence_of_element_located((By.XPATH, "div[@class='product-info']/div[@class='product-image-container']/div[@class='product-image-wrapper']/div[@class='product-image']/div[@class='product-image-content']/div/div")))
            images = df_producto.find_elements("xpath", "div[@class='product-info']/div[@class='product-image-container']/div[@class='product-image-wrapper']/div[@class='product-image']/div[@class='product-image-content']/div/div")
            
            img_url = ''
            px_url = ''

            n_imagenes = len(images)

            for im, i in zip(images, range(n_imagenes)):

                try:

                    url = im.get_attribute("style")

                    url = url[url.index('(')+2:url.index(')')-1]

                    url_img = Request(url, headers={'User-Agent': 'Mozilla/5.0'})

                    u = urlopen(url_img)
                    raw_data = u.read()
                    u.close()

                    img = Image.open(BytesIO(raw_data))

                    if i == n_imagenes-1:
                        img_url = img_url + url
                        px_url = px_url + str(img.size) 
                
                    else:
                        img_url = img_url + url + '; '
                        px_url = px_url + str(img.size) + '; '

                except:
                    
                    if i == n_imagenes-1:

                        img_url = img_url + 'url_image not found'
                        px_url = px_url + ''

                    else:

                        img_url = img_url + 'url_image not found' + '; '
                        px_url = px_url + '; '
            
            image_info_1, image_info_2 = img_url, px_url

        except:
            image_info_1,image_info_2 = '',''

        try:
            first = url_sku[1]
        except:
            first = ''
        
        try:
            second = url_sku[2]
        except:
            second = ''
        
        try:
            third = url_sku[3]
        except:
            third = ''
        
        try:
            category_1 = url_sku[1]
        except:
            category_1 = ''

        try:
            category_2 = url_sku[2]
        except:
            category_2 = ''

    except:
        
        id_url = ''
        name_url = ''
        description_url = ''
        atr_sku = ''
        normal_price = ''
        internet_price = ''
        cmr_price = ''
        id_client_sku = ''
        cod_sku = ''
        image_info_1 = ''
        image_info_2 = ''
        first = ''
        second = ''
        third = ''
        category_1 = ''
        category_2 = ''
    
    driver.close()

    return pais_sku, url_sku[0], id_url, name_url, description_url, id_client_sku, cod_sku, first, second, third, category_1, category_2, atr_sku, normal_price, internet_price, cmr_price, image_info_1, image_info_2

In [ ]:
lista = df_url_sku[['url_product', 'first_level_category', 'second_level_category', 'third_level_category']].values.tolist()
lista

[['https://www.jumbo.cl/carne-molida-5-materia-grasa-500g/p',
  'Supermercado',
  'Exclusivo en Jumbo',
  'Cuisine & Co'],
 ['https://www.jumbo.cl/carne-molida-10-materia-grasa/p',
  'Supermercado',
  'Exclusivo en Jumbo',
  'Cuisine & Co'],
 ['https://www.jumbo.cl/carne-molida-magra-3-grasa-400-gr/p',
  'Supermercado',
  'Exclusivo en Jumbo',
  'Cuisine & Co'],
 ['https://www.jumbo.cl/atun-lomitos-en-agua-104-g-drenado-cuisine-and-co-1828067/p',
  'Supermercado',
  'Exclusivo en Jumbo',
  'Cuisine & Co'],
 ['https://www.jumbo.cl/huevos-extra-grandes-blancos-bandeja-20-un-cuisine-and-co-1863082/p',
  'Supermercado',
  'Exclusivo en Jumbo',
  'Cuisine & Co'],
 ['https://www.jumbo.cl/leche-descremada-1-l-cuisine-and-co-1858323/p',
  'Supermercado',
  'Exclusivo en Jumbo',
  'Cuisine & Co'],
 ['https://www.jumbo.cl/choclo-en-grano-congelado-500-g-cuisine-and-co-1763677/p',
  'Supermercado',
  'Exclusivo en Jumbo',
  'Cuisine & Co'],
 ['https://www.jumbo.cl/bistec-de-posta-rosada-cat-v-gra

In [ ]:
lista[3]

['https://www.jumbo.cl/atun-lomitos-en-agua-104-g-drenado-cuisine-and-co-1828067/p',
 'Supermercado',
 'Exclusivo en Jumbo',
 'Cuisine & Co']

In [ ]:
get_scraping_sku('CL', lista[3])

('CL',
 'https://www.jumbo.cl/atun-lomitos-en-agua-104-g-drenado-cuisine-and-co-1828067/p',
 'Código: 1828067',
 'Atún lomitos en agua 104 g drenado',
 'Lomitos de atún en agua, un producto de la industria ecuatoriana. El atún es una excelente fuente de Omega 3.',
 'Código: 1828067',
 'Código: 1828067',
 'Supermercado',
 'Exclusivo en Jumbo',
 'Cuisine & Co',
 'Supermercado',
 'Exclusivo en Jumbo',
 'Cuisine & Co',
 '',
 '$1.499',
 '',
 'https://jumbo.vtexassets.com/arquivos/ids/705406-750-750?width=750&height=750&aspect=true',
 '(750, 750)')

In [ ]:
def get_df_scraping(pais:str, list_url:list):

    results = Parallel(n_jobs=-1, prefer="threads")(delayed(get_scraping_sku)(pais, li) for li in tqdm.tqdm(list_url))

    return(pd.DataFrame(results, columns=['pais', 'url_sku', 'id', 'name', 'description', 'id_client', 'cod_sku', 'first_category',
                                        'second_category', 'third_category', 'category_1', 'category_2', 'attributes', 'normal_price',
                                        'internet_price', 'cmr_price', 'url_image', 'pixel_image']))

In [ ]:
lista[:5]

[['https://www.jumbo.cl/carne-molida-5-materia-grasa-500g/p',
  'Supermercado',
  'Exclusivo en Jumbo',
  'Cuisine & Co'],
 ['https://www.jumbo.cl/carne-molida-10-materia-grasa/p',
  'Supermercado',
  'Exclusivo en Jumbo',
  'Cuisine & Co'],
 ['https://www.jumbo.cl/carne-molida-magra-3-grasa-400-gr/p',
  'Supermercado',
  'Exclusivo en Jumbo',
  'Cuisine & Co'],
 ['https://www.jumbo.cl/atun-lomitos-en-agua-104-g-drenado-cuisine-and-co-1828067/p',
  'Supermercado',
  'Exclusivo en Jumbo',
  'Cuisine & Co'],
 ['https://www.jumbo.cl/huevos-extra-grandes-blancos-bandeja-20-un-cuisine-and-co-1863082/p',
  'Supermercado',
  'Exclusivo en Jumbo',
  'Cuisine & Co']]

In [ ]:
df_final = get_df_scraping('CL', lista[:5])

100%|██████████| 5/5 [00:00<00:00, 2539.23it/s]


In [ ]:
df_final.head(3)

,pais,url_sku,id,name,description,id_client,cod_sku,first_category,second_category,third_category,category_1,category_2,attributes,normal_price,internet_price,cmr_price,url_image,pixel_image
0,CL,https://www.jumbo.cl/carne-molida-5-materia-gr...,Código: 1670575,Carne molida 5% grasa 500 g,La carne molida 5% grasa es un producto que si...,Código: 1670575,Código: 1670575,Supermercado,Exclusivo en Jumbo,Cuisine & Co,Supermercado,Exclusivo en Jumbo,Cuisine & Co,,,,https://jumbo.vtexassets.com/arquivos/ids/6522...,"(750, 750)"
1,CL,https://www.jumbo.cl/carne-molida-10-materia-g...,Código: 1670574,Carne molida vacuno 10% grasa 500 g,Ideal para hacer variadas preparaciones donde ...,Código: 1670574,Código: 1670574,Supermercado,Exclusivo en Jumbo,Cuisine & Co,Supermercado,Exclusivo en Jumbo,Cuisine & Co,,,,https://jumbo.vtexassets.com/arquivos/ids/6522...,"(750, 750)"
2,CL,https://www.jumbo.cl/carne-molida-magra-3-gras...,Código: 1912535,Carne molida magra 3% grasa 500 g,La carne molida magra 3% grasa es un producto ...,Código: 1912535,Código: 1912535,Supermercado,Exclusivo en Jumbo,Cuisine & Co,Supermercado,Exclusivo en Jumbo,Cuisine & Co,,,,https://jumbo.vtexassets.com/arquivos/ids/6522...,"(750, 750)"
